### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,39903022,77583,79085,51530,0,0,1021,5,2023-02-02 19:30,2023-02-02 19:30,1,2,2085,5,1
34,39902819,10653,79085,3,0,0,1021,6,2023-02-03 21:59,2023-02-03 21:59,1,1,6947,4,1
35,39907428,116340,79085,51530,0,0,1022,0,2023-02-04 21:30,2023-02-04 21:30,2,1,6741,6,1
36,39718551,79085,120761,1345,13,61,1022,1,2023-02-05 21:58,2023-02-05 21:58,3,0,5500,4,1
37,39908292,79085,116340,51530,0,0,1022,2,2023-02-06 21:30,2023-02-06 21:30,0,8,3063,2,1
38,39908521,122834,79085,51531,0,0,1022,3,2023-02-07 22:00,2023-02-07 22:00,0,7,5226,2,1
39,39718535,79085,80120,1345,14,61,1022,4,2023-02-08 21:58,2023-02-08 21:58,0,4,5500,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,77583,30944,35996,0,11,0,0,0,max2,10,8,9,39903022
1,79085,39304,34252,0,7,4,1,0,4-4-2b,7,7,8,39903022
0,10653,32280,37495,0,5,1,0,0,ala,6,8,7,39902819
1,79085,37740,32525,0,9,4,0,0,5-4-1a,6,7,5,39902819
0,116340,17725,39880,3,22,3,1,0,4-3-2-1,10,9,12,39907428
1,79085,52406,30251,4,2,5,0,0,4-4-2b,7,7,8,39907428
0,79085,25504,38779,0,18,1,0,0,4-4-2b,9,7,8,39718551
1,120761,44512,31237,5,6,3,2,0,Attack,6,8,8,39718551
0,79085,46274,35324,0,3,1,0,0,5-4-1b,7,7,8,39908292
1,116340,23764,34714,2,19,0,0,0,4.3.2.1 mod,11,9,10,39908292


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

674

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1345

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,10246,14,33,10,3,1,33,6,33527033010009,1345
1,10520,14,32,10,2,2,45,11,32534045010011,1345
2,48541,14,28,9,1,4,47,34,28513047009007,1345
3,80120,14,27,8,3,3,18,11,27507018008012,1345
4,121094,14,25,8,1,5,24,16,25508024008008,1345
5,79085,14,22,6,4,4,20,20,22500020006005,1345
6,31199,14,18,5,3,6,22,23,18499022005006,1345
7,23755,14,17,5,2,7,20,28,17492020005010,1345
8,88725,14,17,5,2,7,19,27,17492019005001,1345
9,89978,14,7,2,1,11,11,40,7471011002004,1345


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

3

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()